In [1]:
!nvidia-smi

Sat Oct 21 16:34:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.199.02   Driver Version: 470.199.02   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:01:00.0 Off |                  Off |
| 30%   39C    P8    28W / 300W |      3MiB / 48685MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A6000    Off  | 00000000:05:00.0 Off |                  Off |
| 58%   

In [2]:
import os
import torch

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = str(5)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
# !pip install -q bitsandbytes
# !pip install -q transformers 
# !pip install -q peft
# !pip install -q accelerate
# !pip install -q datasets

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
# from peft import PeftModel, PeftConfig

In [5]:
tokenizer = AutoTokenizer.from_pretrained("42dot/42dot_LLM-SFT-1.3B")
model = AutoModelForCausalLM.from_pretrained("42dot/42dot_LLM-SFT-1.3B").to(device)

model.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(50304, 2048, padding_idx=50258)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSN

In [6]:
question = "간단한 문장을 제시하라."
word = "자전거"
prefix_prompt = f'### 입력: {question}\n\n### 답변:'


outputs = []
for idx in range(len(word)):
    char = word[idx]
    
    # encode context the generation is conditioned on
    if len(outputs)==0:
        input_sentence = f'{prefix_prompt}{" ".join(outputs).strip()} {char}'
    else:
        input_sentence = f'{prefix_prompt} {" ".join(outputs).strip()} {char}'
    input_ids = tokenizer.encode(input_sentence, return_tensors='pt').to(device)

    
    if idx == 0:
        beam_output = model.generate(
            input_ids = input_ids, 
            max_length=len(input_ids[0]) + 10,
            early_stopping=True,
            do_sample=True, # 샘플링 전략 사용
            temperature=1.25,
        )
    
        
    elif idx != len(word)-1:
        # set no_repeat_ngram_size to 2
        beam_output = model.generate(
            input_ids = input_ids, 
            max_new_tokens = 100,
            # max_length=len(input_ids[0]) + 10,
            early_stopping=True,
            # num_beams=10, 
            # no_repeat_ngram_size=2, 
            do_sample=True, # 샘플링 전략 사용
            temperature=0.5,
            # top_p=0.95, # 누적 확률이 95%인 후보집합에서만 생성
            # top_k=50, # 확률 순위가 50위 밖인 토큰은 샘플링에서 제외
        )
    
    elif idx == len(word)-1:
        # set no_repeat_ngram_size to 2
        beam_output = model.generate(
            input_ids = input_ids, 
            max_new_tokens = 100,
            # max_length=len(input_ids[0]) + 10,
            early_stopping=True,
            # num_beams=10, 
            # no_repeat_ngram_size=2, 
            do_sample=False, # 샘플링 전략 사용
            # top_p=0.95, # 누적 확률이 95%인 후보집합에서만 생성
            # top_k=50, # 확률 순위가 50위 밖인 토큰은 샘플링에서 제외
            
            
            # input_ids = input_ids, 
            # max_length=len(input_ids[0]) + 50,
            # early_stopping=True,
            # num_beams=10, 
            no_repeat_ngram_size=2, 
            # do_sample=True, # 샘플링 전략 사용
            # top_p=0.95, # 누적 확률이 95%인 후보집합에서만 생성
            # top_k=50, # 확률 순위가 50위 밖인 토큰은 샘플링에서 제외
        )

    output = tokenizer.decode(beam_output[0], skip_special_tokens=True)[len(input_sentence)-1:].split('#')[0].split('. ')[0].strip() # .split('\n')[0]
    outputs.append(output)

    print(f"{char} : {outputs[idx]}")

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


자 : 자라나는 식물은 자라고 있다.
전 : 전 세계적으로 자라는 식물은 약 10억 종에 달한다.
거 : 거미, 벌, 나비, 새 등 다양한 동물들도 자란다.


In [7]:
word = "인하대"
question = f"{word}에 대한 재미있고 간단한 문장을 말씀드리겠습니다."
prefix_prompt = f'{question} '


outputs = []
for idx in range(len(word)):
    char = word[idx]
    
    # encode context the generation is conditioned on
    if len(outputs)==0:
        input_sentence = f'{prefix_prompt}{" ".join(outputs).strip()} {char}'
    else:
        input_sentence = f'{prefix_prompt} {" ".join(outputs).strip()} {char}'
    input_ids = tokenizer.encode(input_sentence, return_tensors='pt').to(device)

    
    if idx == 0:
        beam_output = model.generate(
            input_ids = input_ids, 
            max_new_tokens = 100,
            early_stopping=True,
            do_sample=True, # 샘플링 전략 사용
            temperature=0.5,
        )
    
        
    elif idx != len(word)-1:
        # set no_repeat_ngram_size to 2
        beam_output = model.generate(
            input_ids = input_ids, 
            max_new_tokens = 100,
            early_stopping=True,
            no_repeat_ngram_size=2, 
            do_sample=False, # 샘플링 전략 사용
            temperature=0.25,
            # top_p=0.95, # 누적 확률이 95%인 후보집합에서만 생성
            # top_k=50, # 확률 순위가 50위 밖인 토큰은 샘플링에서 제외
        )
    
    elif idx == len(word)-1:
        # set no_repeat_ngram_size to 2
        beam_output = model.generate(
            input_ids = input_ids, 
            # max_new_tokens = 100,
            max_length=len(input_ids[0]) + 100,
            early_stopping=True,
            num_beams=10, 
            no_repeat_ngram_size=2, 
            do_sample=True, # 샘플링 전략 사용
            temperature=1,
            top_p=0.90, # 누적 확률이 95%인 후보집합에서만 생성
            # top_k=50, # 확률 순위가 50위 밖인 토큰은 샘플링에서 제외
            
        )

        
    output = tokenizer.decode(beam_output[0], skip_special_tokens=True)[len(input_sentence)-1:].split('\n')[0].split('#')[0]
    if '.' in output:
        output = output.split('.')[0] + '.'
    if ',' in output:
        output = output.split(',')[0] + ','

        
    outputs.append(output)

    print(f"{char} : {outputs[idx]}")

인 : 인하대 학생들이여,
하 : 하하하!


/usr/local/lib/python3.8/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.25` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


대 : 대동소이하다.


In [8]:
word = "인하대"
question = f"{word}에 대한 재미있고 간단한 문장을 말씀드리겠습니다."
prefix_prompt = f'{question} '


outputs = []
for idx in range(len(word)):
    char = word[idx]
    
    # encode context the generation is conditioned on
    if len(outputs)==0:
        input_sentence = f'{prefix_prompt}{" ".join(outputs).strip()} {char}'
    else:
        input_sentence = f'{prefix_prompt} {" ".join(outputs).strip()} {char}'
    input_ids = tokenizer.encode(input_sentence, return_tensors='pt').to(device)

    
    if idx == 0:
        beam_output = model.generate(
            input_ids = input_ids, 
            max_new_tokens = 100,
            early_stopping=True,
            do_sample=True, # 샘플링 전략 사용
            temperature=0.5,
        )
    
        
    elif idx != len(word)-1:
        # set no_repeat_ngram_size to 2
        beam_output = model.generate(
            input_ids = input_ids, 
            max_new_tokens = 100,
            early_stopping=True,
            no_repeat_ngram_size=2, 
            do_sample=False, # 샘플링 전략 사용
            temperature=0.25,
            # top_p=0.95, # 누적 확률이 95%인 후보집합에서만 생성
            # top_k=50, # 확률 순위가 50위 밖인 토큰은 샘플링에서 제외
        )
    
    elif idx == len(word)-1:
        # set no_repeat_ngram_size to 2
        beam_output = model.generate(
            input_ids = input_ids, 
            # max_new_tokens = 100,
            max_length=len(input_ids[0]) + 100,
            early_stopping=True,
            num_beams=10, 
            no_repeat_ngram_size=2, 
            do_sample=True, # 샘플링 전략 사용
            temperature=1,
            top_p=0.90, # 누적 확률이 95%인 후보집합에서만 생성
            # top_k=50, # 확률 순위가 50위 밖인 토큰은 샘플링에서 제외
            
        )

        
    output = tokenizer.decode(beam_output[0], skip_special_tokens=True)[len(input_sentence)-1:].split('\n')[0].split('#')[0]
    if '.' in output:
        output = output.split('.')[0] + '.'
    if ',' in output:
        output = output.split(',')[0] + ','

        
    outputs.append(output)

    print(f"{char} : {outputs[idx]}")

인 : 인하대 근처에 있는 카페에서 친구와 함께 산책을 하며 커피를 마시는데,
하 : 하얀 눈이 내리는 날이었다.
대 : 대자연의 아름다움을 만끽할 수 있었다.


In [9]:
word = "이채은"
question = f"{word}에 대한 재미있고 간단한 문장을 말씀드리겠습니다."
prefix_prompt = f'{question} '


outputs = []
for idx in range(len(word)):
    char = word[idx]
    
    # encode context the generation is conditioned on
    if len(outputs)==0:
        input_sentence = f'{prefix_prompt}{" ".join(outputs).strip()} {char}'
    else:
        input_sentence = f'{prefix_prompt} {" ".join(outputs).strip()} {char}'
    input_ids = tokenizer.encode(input_sentence, return_tensors='pt').to(device)

    
    if idx == 0:
        beam_output = model.generate(
            input_ids = input_ids, 
            max_new_tokens = 100,
            early_stopping=True,
            num_beams=10, 
            no_repeat_ngram_size=2, 
            do_sample=True, # 샘플링 전략 사용
            temperature=0.5,
            top_p=0.95, # 누적 확률이 95%인 후보집합에서만 생성
        )
    
        
    elif idx != len(word)-1:
        # set no_repeat_ngram_size to 2
        beam_output = model.generate(
            input_ids = input_ids, 
            max_new_tokens = 100,
            early_stopping=True,
            no_repeat_ngram_size=2, 
            num_beams=10, 
            do_sample=False, # 샘플링 전략 사용
            # temperature=0.25,
            # top_p=0.95, # 누적 확률이 95%인 후보집합에서만 생성
            # top_k=50, # 확률 순위가 50위 밖인 토큰은 샘플링에서 제외
        )
    
    elif idx == len(word)-1:
        # set no_repeat_ngram_size to 2
        beam_output = model.generate(
            input_ids = input_ids, 
            # max_new_tokens = 100,
            max_length=len(input_ids[0]) + 100,
            early_stopping=True,
            num_beams=10, 
            no_repeat_ngram_size=2, 
            do_sample=True, # 샘플링 전략 사용
            temperature=1,
            top_p=0.90, # 누적 확률이 95%인 후보집합에서만 생성
            # top_k=50, # 확률 순위가 50위 밖인 토큰은 샘플링에서 제외
            
        )

        
    output = tokenizer.decode(beam_output[0], skip_special_tokens=True)[len(input_sentence)-1:].split('\n')[0].split('#')[0]
    if '.' in output:
        output = output.split('. ')[0] + '.'
    # if ',' in output:
        # output = output.split(', ')[0] + ','

        
    outputs.append(output)

    print(f"{char} : {outputs[idx]}")

이 : 이채연은 귀여운 외모와 밝은 성격으로 많은 사람들의 사랑을 받았습니다..
채 : 채연이는 항상 웃는 얼굴로 사람들을 행복하게 만들었습니다..
은 : 은은한 미소와 상냥한 말투로 사람들의 마음을 편안하게 해주었습니다..


In [13]:
word = "트와이스"
question = f"{word}에 대한 재미있고 간단한 문장을 말씀드리겠습니다."
prefix_prompt = f'{question} '


outputs = []
for idx in range(len(word)):
    char = word[idx]
    
    # encode context the generation is conditioned on
    if len(outputs)==0:
        input_sentence = f'{prefix_prompt}{" ".join(outputs).strip()} {char}'
    else:
        input_sentence = f'{prefix_prompt} {" ".join(outputs).strip()} {char}'
    input_ids = tokenizer.encode(input_sentence, return_tensors='pt').to(device)

    
    if idx == 0:
        beam_output = model.generate(
            input_ids = input_ids, 
            max_new_tokens = 100,
            early_stopping=True,
            do_sample=True, # 샘플링 전략 사용
            temperature=0.5,
        )
    
        
    elif idx != len(word)-1:
        # set no_repeat_ngram_size to 2
        beam_output = model.generate(
            input_ids = input_ids, 
            max_new_tokens = 100,
            early_stopping=True,
            no_repeat_ngram_size=2, 
            do_sample=False, # 샘플링 전략 사용
            temperature=0.25,
            # top_p=0.95, # 누적 확률이 95%인 후보집합에서만 생성
            # top_k=50, # 확률 순위가 50위 밖인 토큰은 샘플링에서 제외
        )
    
    elif idx == len(word)-1:
        # set no_repeat_ngram_size to 2
        beam_output = model.generate(
            input_ids = input_ids, 
            # max_new_tokens = 100,
            max_length=len(input_ids[0]) + 100,
            early_stopping=True,
            num_beams=10, 
            no_repeat_ngram_size=2, 
            do_sample=True, # 샘플링 전략 사용
            temperature=1,
            top_p=0.90, # 누적 확률이 95%인 후보집합에서만 생성
            # top_k=50, # 확률 순위가 50위 밖인 토큰은 샘플링에서 제외
            
        )

        
    output = tokenizer.decode(beam_output[0], skip_special_tokens=True)[len(input_sentence)-1:].split('\n')[0].split('#')[0]
    if '.' in output:
        output = output.split('. ')[0] + '.'
    # if ',' in output:
        # output = output.split(',')[0] + ','

        
    outputs.append(output)

    print(f"{char} : {outputs[idx]}")

트 : 트와이스는 정말로 대단한 그룹이에요!
와 : 와, 트와이이스의 노래는 항상 최고에요! 트위치에서 트위스트를 치면 트과이스가 춤을 추는 영상이 나와요!
이 : 이 그룹은 정말 멋진 무대 퍼포먼스를 보여줘요.
스 : 스윗한 미소와 귀여운 외모로 전 세계 팬들을 사로잡았어요..
